In [3]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Configuración del servidor Flask
app = Flask(__name__)

# Habilitar CORS
CORS(app)

# Configura el URI del servidor MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")  # Cambia según tu servidor MLflow
model_name = "Tiempo"  # Nombre del modelo registrado
model_version = "14"  # Versión del modelo en MLflow

# Cargar el modelo desde el registro de MLflow
try:
    model_uri = f"models:/{model_name}/{model_version}"
    print(f"Cargando el modelo desde {model_uri}...")
    model = mlflow.sklearn.load_model(model_uri)
    print("Modelo cargado exitosamente.")

    # Inicializar StandardScaler con los datos de entrenamiento (simulación)
    sample_data = pd.DataFrame({
        "distancia_km": [5, 10, 15, 20, 25],
        "numero_articulos": [1, 3, 5, 2, 4],
        "velocidad_kmh": [30, 40, 35, 25, 45]
    })
    scaler = StandardScaler()
    scaler.fit(sample_data)  # Ajustar el escalador con datos de entrenamiento

except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    model = None

@app.route("/", methods=["GET"])
def home():
    return jsonify({"message": "API para estimar tiempo de entrega funcionando correctamente."})

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if model is None:
            return jsonify({'error': 'Modelo no cargado.'}), 500

        # Obtener datos en formato JSON
        data = request.get_json()

        # Validar formato
        if not data or not isinstance(data, list):
            return jsonify({'error': 'Se esperaba una lista de instancias JSON.'}), 400

        # Crear DataFrame con los datos recibidos
        input_data = pd.DataFrame(data)

        # Validar que las columnas requeridas estén presentes
        required_columns = ["distancia_km", "numero_articulos", "velocidad_kmh"]
        if not all(col in input_data.columns for col in required_columns):
            return jsonify({'error': f'Faltan columnas requeridas: {required_columns}'}), 400

        # Imprimir los datos recibidos para depuración
        print("Datos recibidos para predicción:", input_data)
        print("Columnas de los datos recibidos:", input_data.columns.tolist())

        # Preprocesar los datos (escalado) manteniendo los nombres de las columnas
        input_scaled = pd.DataFrame(scaler.transform(input_data), columns=input_data.columns)

        # Realizar predicciones
        predictions = model.predict(input_scaled)

        return jsonify({'predictions': predictions.tolist()})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5001, debug=True, use_reloader=False)


Cargando el modelo desde models:/Tiempo/14...
Modelo cargado exitosamente.
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [29/Jan/2025 00:44:24] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2025 00:44:25] "POST /predict HTTP/1.1" 200 -


Datos recibidos para predicción:    distancia_km  numero_articulos  velocidad_kmh
0             5                 1             30
1            10                 3             40
Columnas de los datos recibidos: ['distancia_km', 'numero_articulos', 'velocidad_kmh']


127.0.0.1 - - [29/Jan/2025 00:48:15] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2025 00:48:15] "POST /predict HTTP/1.1" 200 -


Datos recibidos para predicción:    distancia_km  numero_articulos  velocidad_kmh
0            10                 1             30
1            10                 3             40
Columnas de los datos recibidos: ['distancia_km', 'numero_articulos', 'velocidad_kmh']


...

Datos recibidos para predicción:    distancia_km  numero_articulos  velocidad_kmh
0             5                 1             30
1            10                 3             40
2            15                 5             35
3            20                 2             25
Columnas de los datos recibidos: ['distancia_km', 'numero_articulos', 'velocidad_kmh']


127.0.0.1 - - [29/Jan/2025 01:02:54] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2025 01:02:54] "POST /predict HTTP/1.1" 200 -


Datos recibidos para predicción:    distancia_km  numero_articulos  velocidad_kmh
0             5                 1             30
1            10                 3             40
2            15                 5             35
3            20                 2             25
Columnas de los datos recibidos: ['distancia_km', 'numero_articulos', 'velocidad_kmh']


127.0.0.1 - - [29/Jan/2025 01:02:55] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2025 01:02:55] "POST /predict HTTP/1.1" 200 -


Datos recibidos para predicción:    distancia_km  numero_articulos  velocidad_kmh
0             5                 1             30
1            10                 3             40
2            15                 5             35
3            20                 2             25
Columnas de los datos recibidos: ['distancia_km', 'numero_articulos', 'velocidad_kmh']
